# Part 1: Double Lasso and DAGs (20 points)

In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "GLM", "MLJLinearModels", "MLJBase"])

   Resolving package versions...
  No Changes to `C:\Users\KARL\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\KARL\.julia\environments\v1.11\Manifest.toml`


In [2]:
using CSV
using DataFrames
using GLM
using MLJLinearModels
using MLJBase

In [3]:
file_path = "F:/Programas/GitHub Desktop/CausalAI-Course/data/wage2015_subsample_inference.csv"
data = CSV.read(file_path, DataFrame)

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


1. Consider the US census data from the year 2015 to analyse the effect of college graduate (clg) status
and it’s interaction effects with gender (sex), location (mw, so,we, ne) and both on wage jointly. All
other variables denote some other socio-economic characteristics, e.g. marital status, occupation, and
experience.

In [4]:
categorical_vars = [:sex, :mw, :so, :we, :ne]

for var in categorical_vars
    data[!, Symbol(string(var, "_dummy"))] = convert(Vector{Int}, data[!, var] .== 1)
end

for i in 1:length(categorical_vars)
    for j in i+1:length(categorical_vars)
        interaction_col = Symbol(string(categorical_vars[i], "_x_", categorical_vars[j]))
        # Multiplicación de las dummies convertidas
        data[!, interaction_col] = data[!, Symbol(string(categorical_vars[i], "_dummy"))] .* data[!, Symbol(string(categorical_vars[j], "_dummy"))]
    end
end

first(data, 5)

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,sex_dummy,mw_dummy,so_dummy,we_dummy,ne_dummy,sex_x_mw,sex_x_so,sex_x_we,sex_x_ne,mw_x_so,mw_x_we,mw_x_ne,so_x_we,so_x_ne,we_x_ne
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0


- Use the double lasso technique to find the effect of the treatment and it’s relevant interactions on the wage. To tune the penalization parameter in the lasso step, cross-validate it. (4 pts)

In [ ]:
using Pkg
Pkg.add(["MLJ"])
using MLJ

In [ ]:
X = select(data, Not(:wage))  
y = data.wage

X_matrix = MLJBase.matrix(X)

lasso_model = LassoPath()
lasso_fit = machine(lasso_model, X_matrix, y)

r = range(lasso_model, :alpha, lower=0.01, upper=1.0, length=10)  
cv = CV(nfolds=5, shuffle=true, rng=42)
tuned_model = TunedModel(model=lasso_model, resampling=cv, range=r, measure=rms)
mach = machine(tuned_model, X_matrix, y)
fit!(mach)

report(mach)

In [ ]:
- Report a summary of the estimation of the parameters of interest. (2 pts)

In [ ]:
fitted_model = fitted_params(mach).best_model
coefficients = fitted_model.coefs

for (name, value) in zip(names(X), coefficients)
    println("Variable: ", name, ", Coeficiente: ", value)
end


- Interpret your results. In which group does the college graduate status has more impact over the wage? (2 pts)